In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import json
import geopandas as gpd
from shapely.wkt import loads
from pyproj import Proj, Transformer

In [5]:

def lat_lon_to_axis(lat, lon, axis_direction=(1, 0), crs="EPSG:3857"):
    """
    Convert latitude and longitude to a one-dimensional value in meters along an axis.

    Args:
        lat (float): Latitude of the point.
        lon (float): Longitude of the point.
        axis_direction (tuple): Direction vector of the axis (x, y).
        crs (str): CRS for projecting to meters (default: EPSG:3857).

    Returns:
        float: One-dimensional projection value in meters.
    """
    
    # Define the transformer for projecting lat/lon to the desired CRS
    transformer = Transformer.from_crs("EPSG:4326", crs, always_xy=True)
    
    # Project the latitude and longitude to meters
    x, y = transformer.transform(lon, lat)
    
    # Normalize the axis direction vector
    axis_vector = np.array(axis_direction)
    axis_unit_vector = axis_vector / np.linalg.norm(axis_vector)
    
    # Compute the projection of the point onto the axis
    projection_value = np.dot([x, y], axis_unit_vector)
    return projection_value

def find_traffic_light_id(row, segments_gdf):
    """
    Find the nearest segment ID to a given row in a DataFrame.

    Args:
        row (pd.Series): Row in a DataFrame with columns 'lat' and 'lon'.
        segments_gdf (gpd.GeoDataFrame): GeoDataFrame with segment geometries.

    Returns:
        int: ID of the nearest segment.
    """
    # Create a GeoDataFrame for the point
    point = gpd.GeoDataFrame(
        geometry=[loads(f"POINT ({row['lat']} {row['lon']})")], crs="EPSG:4326" # Make sure lat and lon is given completely right! Lat first then lon
    )
    
    # Reproject both point and segments to a projected CRS for accurate distance calculations
    projected_crs = "EPSG:4326"  # Common projected CRS for distance calculations
    point_proj = point.to_crs(projected_crs)
    segments_gdf_proj = segments_gdf.to_crs(projected_crs)
    max_distance = float("inf")
    nearest_segment_id = None
    for index, row in segments_gdf_proj.iterrows():
        distance = row.geometry.distance(point_proj.geometry[0])
        if distance < max_distance:
            max_distance = distance
            nearest_segment_id = index
    return nearest_segment_id


In [6]:
# loading the main dataframe
main_df = pd.read_csv("20181024_d1_0830_0900_traffic_segmented.csv")
# loading the traffic light states
with open("20181024_d1_0830_0900_traffic_light_states.json") as f:
    traffic_info = json.load(f)
# loading the segments
segments_gdf = gpd.read_file("20181024_d1_0830_0900_traffic_lights_segments.csv")
# Convert to GeoDataFrame
if "geometry" in segments_gdf.columns:
    # Convert 'geometry' to shapely objects if needed
    segments_gdf["geometry"] = segments_gdf["geometry"].apply(loads)
    # Create GeoDataFrame and set the CRS to WGS 84 (latitude/longitude)
    segments_gdf = gpd.GeoDataFrame(segments_gdf, geometry="geometry", crs="EPSG:4326")
else:
    raise ValueError("The DataFrame does not have a 'geometry' column.")

# Reproject to a projected CRS (replace EPSG code with appropriate UTM zone)
segments_gdf = segments_gdf.to_crs("EPSG:25832")  # Example for UTM Zone 32N

# Calculate lengths
segments_gdf["length"] = segments_gdf.geometry.length
segments_gdf = segments_gdf.to_crs("EPSG:4326")  # Example for UTM Zone 32N

lon_one_axis_trajectory = []
lat_one_axis_trajectory = []
for i, row in segments_gdf.iterrows():
    lat_one_axis_trajectory.append(lat_lon_to_axis(row.geometry.coords.xy[0][1], row.geometry.coords.xy[0][0], crs="EPSG:4326"))
    lon_one_axis_trajectory.append(lat_lon_to_axis(row.geometry.coords.xy[1][1], row.geometry.coords.xy[1][0], crs="EPSG:4326"))

segments_gdf["lon_one_axis_trajectory"] = lon_one_axis_trajectory
segments_gdf["lat_one_axis_trajectory"] = lat_one_axis_trajectory

# Calculate total length
total_length = segments_gdf["length"].sum()

# loading the traffic lights
traffic_lights_df = pd.read_csv("traffic_lights.csv")
# find traffic light segment id
traffic_lights_df["nearest_segment_id"] = traffic_lights_df.apply(find_traffic_light_id, axis=1, segments_gdf=segments_gdf)
traffic_lights_df = traffic_lights_df.iloc[:5, :]
# loading the traffic light states
with open("traffic_info_dict.pkl", "rb") as f:
    traffic_lights_dict_states = pickle.load(f)


In [7]:
# Read the CSV file
main_df_truncated_with_cell = pd.read_csv("20181024_d1_0830_0900_segmented_oneaxistrajectory_cell.csv")

# Group by the desired columns and aggregate the vehicle IDs into a frozenset
grouped_with_veh_ids = main_df_truncated_with_cell.groupby(["link_id", "time"]).agg({
    "veh_id": lambda x: frozenset(x)  # Use frozenset instead of set
}).reset_index()

# Merge the new column back into the original dataframe
main_df_truncated_with_cell = main_df_truncated_with_cell.merge(grouped_with_veh_ids, on=["link_id", "time"], suffixes=('', '_list'))

# Rename the new column for clarity
main_df_truncated_with_cell.rename(columns={"veh_id_list": "veh_id_list"}, inplace=True)

# Display the shapes
duplicate_dropped = main_df_truncated_with_cell.drop_duplicates(subset=["link_id", "time", "veh_id_list"])[["veh_id_list", "link_id", "time"]]


link_dataframes = {}
for segment_id in duplicate_dropped["link_id"].unique():
    
    segment_df = duplicate_dropped[duplicate_dropped["link_id"] == segment_id].reset_index().drop(columns=["index"])
    segment_df = segment_df.sort_values("time")

    segment_df["N_down"] = segment_df["veh_id_list"] - segment_df["veh_id_list"].shift(1)
    segment_df["N_up"] = segment_df["veh_id_list"].shift(-1) - segment_df["veh_id_list"] 

    segment_df["N_cumsum_down"] = segment_df["N_down"].iloc[1:].apply(len).cumsum()
    segment_df["N_cumsum_up"] = segment_df["N_up"].iloc[:-1].apply(len).cumsum()
    
    link_dataframes[segment_id] = segment_df


In [32]:
import math
def cumulativecount_up(time, segment_id):
    segment_df = link_dataframes[segment_id]
    closest_time = segment_df[segment_df['time'] <= time].time.max()
    if math.isnan(closest_time):
        return 0
    return segment_df[segment_df['time'] == closest_time]["N_cumsum_up"].values[0]

def cumulativecount_down(time, segment_id):
    segment_df = link_dataframes[segment_id]
    closest_time = segment_df[segment_df['time'] <= time].time.max()
    if math.isnan(closest_time):
        return 0
    return segment_df[segment_df['time'] == closest_time]["N_cumsum_down"].values[0]

In [65]:
from constants import CTMParameters
from helper import *
import math

# Notes for Maziar:
## Do not use CTMParameters().object at all! If you use it, it will create a new object every time you call it. Instead, use the instance that you make of it!
## Try avoidng using the istance of CTMParemeters as a global object. Instead, pass it as an argument to the functions that need it.
## The docstring could be formatted for clarity: 1- Mention all parameters and their expected types! 2- Include an explanation of the function's output!
## Check for invalid or empty densities.
## Use descriptive variable names for better readability.
## Avoid magic numbers like 1 for the green light status; use constants instead.


# load params
ctm_params = CTMParameters() # for now, we use same parameters we had in CTM model
ctm_params.segment_length = segments_gdf["length"].mean()
# define the function for finding cumulative counts
N_upstr = cumulativecount_up
N_downstr = cumulativecount_down

## Cell transmission model: update cell density
# time: current simulation time
# arguments: N_upstrs: cumulative count of vehicles in upstream at a given time. I'll pass N_downstr(t) and it should tell me the cumulative count
# N_downstr: cumulative count at link downstream at time t (current time)
# entry_flow: for now constant, entering to the link in vehicles per second
# assumptions: jam density and max flow are constant for all cells
def update_point_queue(time, segment_id, entry_flow, traffic_lights_df, traffic_lights_dict_states):

    # check if there is a traffic light at the end of the segment
    if is_tl(segment_id, traffic_lights_df):
        # check the status of the traffic light
        if tl_status(time, segment_id, traffic_lights_df, traffic_lights_dict_states) == 1: # green light
            # find the link sending flow using point queue model 
            sending_flow = min( N_upstr(time + ctm_params.time_step - (ctm_params.segment_length/ctm_params.free_flow_speed), segment_id) - N_downstr(time + ctm_params.time_step - (ctm_params.segment_length/ctm_params.free_flow_speed), segment_id), ctm_params.max_flow()*ctm_params.time_step)
        else:
            sending_flow = 0
    else: # no traffic light at the end of the link
        sending_flow = min( N_upstr(time + ctm_params.time_step - (ctm_params.segment_length/ctm_params.free_flow_speed), segment_id) - N_downstr(time + ctm_params.time_step - (ctm_params.segment_length/ctm_params.free_flow_speed), segment_id), ctm_params.max_flow()*ctm_params.time_step)

    # find the number of vehicles in the link at the next time step
    n_current = N_downstr(time, segment_id) - N_upstr(time, segment_id)   # current number of vehicles
    n_updated = n_current + entry_flow * ctm_params.time_step -  sending_flow 

    link_outflow = sending_flow
    link_density = n_updated / ctm_params.segment_length
    return link_density, link_outflow    # please note the   order ## I will:)


# Get inflow

In [66]:
inflow = pd.read_csv("20181024_d1_0830_0900_inflow.csv")

# Running the model

In [67]:
from collections import defaultdict

point_queue_data = defaultdict(dict)

for segment_id in duplicate_dropped["link_id"].unique():
    segment_df = duplicate_dropped[duplicate_dropped["link_id"] == segment_id].reset_index().drop(columns=["index"])
    segment_df = segment_df.sort_values("time")
    for time in segment_df["time"].unique():
            
        entry_flow = inflow[(inflow["link_id"] == segment_id) & (inflow["time"] == time)]["inflow"]
        if entry_flow.empty:
            entry_flow = 0
        else:
            entry_flow = entry_flow.values[0]
        
        link_density, link_outflow = update_point_queue(time, segment_id, entry_flow, traffic_lights_df, traffic_lights_dict_states)
        
        point_queue_data[segment_id][time] = {"link_density": link_density, "link_outflow": link_outflow}
        print(time, segment_id, "done")
        

0.0 4 done
0.04 4 done
0.08 4 done
0.12 4 done
0.16 4 done
0.2 4 done
0.24 4 done
0.28 4 done
0.32 4 done
0.36 4 done
0.4 4 done
0.44 4 done
0.48 4 done
0.52 4 done
0.56 4 done
0.6 4 done
0.64 4 done
0.68 4 done
0.72 4 done
0.76 4 done
0.8 4 done
0.84 4 done
0.88 4 done
0.92 4 done
0.96 4 done
1.0 4 done
1.04 4 done
1.08 4 done
1.12 4 done
1.16 4 done
1.2 4 done
1.24 4 done
1.28 4 done
1.32 4 done
1.36 4 done
1.4 4 done
1.44 4 done
1.48 4 done
1.52 4 done
1.56 4 done
1.6 4 done
1.64 4 done
1.68 4 done
1.72 4 done
1.76 4 done
1.8 4 done
1.84 4 done
1.88 4 done
1.92 4 done
1.96 4 done
2.0 4 done
2.04 4 done
2.08 4 done
2.12 4 done
2.16 4 done
2.2 4 done
2.24 4 done
2.28 4 done
2.32 4 done
2.36 4 done
2.4 4 done
2.44 4 done
2.48 4 done
2.52 4 done
2.56 4 done
2.6 4 done
2.64 4 done
2.68 4 done
2.72 4 done
2.76 4 done
2.8 4 done
2.84 4 done
2.88 4 done
2.92 4 done
2.96 4 done
3.0 4 done
3.04 4 done
3.08 4 done
3.12 4 done
3.16 4 done
3.2 4 done
3.24 4 done
3.28 4 done
3.32 4 done
3.36 4 do

In [47]:
import pickle

with open("20181024_d1_0830_0900_result_point_queue.pkl", "wb") as f:
    pickle.dump(point_queue_data, f)


point_queue_data = pickle.load(open("20181024_d1_0830_0900_result_point_queue.pkl", "rb"))


In [76]:
duplicate_dropped["no_veh"] = duplicate_dropped["veh_id_list"].apply(len)
duplicate_dropped["density"] = duplicate_dropped["no_veh"] / ctm_params.segment_length
duplicate_dropped

,veh_id_list,link_id,time,no_veh,density
0,"(1, 59, 60, 84)",4,0.00,4,0.034735
1,"(1, 59, 60, 84)",4,0.04,4,0.034735
2,"(1, 59, 60, 84)",4,0.08,4,0.034735
3,"(1, 59, 60, 84)",4,0.12,4,0.034735
4,"(1, 59, 60, 84)",4,0.16,4,0.034735
...,...,...,...,...,...
694918,"(899, 903, 904, 906, 908, 909, 911, 913, 914, ...",0,818.64,12,0.104205
694919,"(899, 903, 904, 906, 908, 909, 911, 913, 914, ...",0,818.68,12,0.104205
694920,"(899, 903, 904, 906, 908, 909, 911, 913, 914, ...",0,818.72,12,0.104205
694921,"(899, 903, 904, 906, 908, 909, 911, 913, 914, ...",0,818.76,12,0.104205


# Checking the point queue model

In [80]:
duplicate_dropped["predicted_density"] = duplicate_dropped.apply(lambda x: point_queue_data[x["link_id"]][x["time"]]["link_density"], axis=1).fillna(0)
duplicate_dropped

,veh_id_list,link_id,time,no_veh,density,predicted_density
0,"(1, 59, 60, 84)",4,0.00,4,0.034735,0.0
1,"(1, 59, 60, 84)",4,0.04,4,0.034735,0.0
2,"(1, 59, 60, 84)",4,0.08,4,0.034735,0.0
3,"(1, 59, 60, 84)",4,0.12,4,0.034735,0.0
4,"(1, 59, 60, 84)",4,0.16,4,0.034735,0.0
...,...,...,...,...,...,...
694918,"(899, 903, 904, 906, 908, 909, 911, 913, 914, ...",0,818.64,12,0.104205,0.0
694919,"(899, 903, 904, 906, 908, 909, 911, 913, 914, ...",0,818.68,12,0.104205,0.0
694920,"(899, 903, 904, 906, 908, 909, 911, 913, 914, ...",0,818.72,12,0.104205,0.0
694921,"(899, 903, 904, 906, 908, 909, 911, 913, 914, ...",0,818.76,12,0.104205,0.0


# RMSE for each link

In [81]:
for link_id in duplicate_dropped["link_id"].unique():
    rmse = np.sqrt(np.mean((duplicate_dropped[duplicate_dropped["link_id"] == link_id]["density"] - duplicate_dropped[duplicate_dropped["link_id"] == link_id]["predicted_density"])**2))
    print("For link with id: ", link_id, " the RMSE is: ", rmse)

For link with id:  4  the RMSE is:  0.03651251539881699
For link with id:  3  the RMSE is:  0.07388087888792845
For link with id:  2  the RMSE is:  0.10950367177654731
For link with id:  1  the RMSE is:  0.05338863019011068
For link with id:  0  the RMSE is:  0.07263397852867445
